In [11]:
import pandas as pd
import numpy as np
import nltk
import spacy
import os
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.fillna('strange',inplace = True)
df_test.fillna('strange',inplace = True)

## Defining the POS 

In [6]:


df_train['from'] = 'train'
df_test['from'] = 'test'
df = pd.concat([df_train,df_test])

df.head()

nlp = spacy.load('en_core_web_sm')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


## Test POS Tagging 
## Combined then to sentence and then tagged the POS

In [ ]:
import time
def getting_the_pos(data):
    try:
        sentence = data["Word"]
        result = (' '.join(list(sentence)))
        doc = nlp(result)
        count = 0
        for token in (doc):
            data.loc[data['Word']==token.text,'POS'] =  token.pos_
            data.loc[data['Word']==token.text,'tag_pos'] = token.tag_
    except:
        pass
    return(data)
from tqdm import tqdm
for i in tqdm(range(0,30)):
    df = df_test[100000*i:100000*(i+1)]
    start_time = time.time()
    driving_at_ninety = pd.DataFrame()
    driving_at_ninety = pd.concat([driving_at_ninety,df.groupby(['Doc_ID']).apply(getting_the_pos)])
    driving_at_ninety.to_csv("output_pos_tag_train/test_{0}_se_{1}_tak.csv".format(1000000*i,1000000*(i+1)))
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import time
def getting_the_pos(data):
    sentence = data["Word"]
    result = (' '.join(list(sentence)))
    doc = nlp(result)
    count = 0
    for token in (doc):
        data.loc[data['Word']==token.text,'POS'] =  token.pos_
        data.loc[data['Word']==token.text,'tag_pos'] = token.tag_
    return(data)
from tqdm import tqdm
for i in tqdm(range(0,46)):
    df = df_train[100000*i:100000*(i+1)]
    start_time = time.time()
    driving_at_ninety = pd.DataFrame()
    driving_at_ninety = pd.concat([driving_at_ninety,df.groupby(['Doc_ID']).apply(getting_the_pos)])
    driving_at_ninety.to_csv("output_pos_tag_train/{0}_se_{1}_tak.csv".format(1000000*i,1000000*(i+1)))
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
train_all = pd.DataFrame()
for i in range(0,46):
    train_all = pd.concat([train_all,pd.read_csv('output_pos_tag_train/{}_se_{}_tak.csv'.format(1000000*i,1000000*(i+1)))])
train_all.set_index('id',inplace = True)
train_all.drop(columns = ['POS_nltk','Unnamed: 0','Unnamed: 0.1'],inplace = True)
test_all.to_csv('full_train_with_pos.csv')

In [ ]:
test_all = pd.DataFrame()
for i in range(0,30):
    test_all = pd.concat([test_all,pd.read_csv('output_pos_tag_train/test_{}_se_{}_tak.csv'.format(1000000*i,1000000*(i+1)))])
test_all.set_index('id',inplace = True)
train_all.drop(columns = ['Unnamed: 0'],inplace = True)
test_all.to_csv('full_test_with_pos.csv') 

In [12]:
train = pd.read_csv('full_train_with_pos.csv')
test = pd.read_csv('full_test_with_pos.csv')

In [8]:
with open('train_ner.txt','w') as file:
    for i in range(train.shape[0]):
        file.write(train['Word'][i]+'\t'+train['tag'][i])

In [13]:
train.fillna('thing',inplace = True)
test.fillna('thing',inplace = True)

In [ ]:
train['tag'].value_counts()

In [11]:
train['tag'].replace(['B-indications','I-indications'],'indications',inplace = True)

In [14]:
train['tag'].value_counts()

O                4446206
B-indications      53003
I-indications      44624
Name: tag, dtype: int64

In [13]:
train.shape

(4543833, 7)

In [14]:
test.shape

(2994463, 6)

In [15]:
train.head()

,id,Doc_ID,POS,Sent_ID,Word,tag,tag_pos
0,1,1,NOUN,1,Obesity,O,NN
1,2,1,ADP,1,in,O,IN
2,3,1,PROPN,1,Low-,O,NNP
3,4,1,CCONJ,1,and,O,CC
4,5,1,thing,1,Middle-Income,O,thing


In [16]:
test.head()

,id,Doc_ID,Sent_ID,Word,POS,tag_pos
0,4543834,30001,191283,CCCVA,PROPN,NNP
1,4543835,30001,191283,",",PUNCT,","
2,4543836,30001,191283,MANOVA,PROPN,NNP
3,4543837,30001,191283,",",PUNCT,","
4,4543838,30001,191283,my,ADJ,PRP$


In [15]:
test['tag'] = 'M'

In [16]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w,p,t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["tag_pos"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("Sent_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s

In [17]:
getter_df_train = SentenceGetter(train)
getter_df_test = SentenceGetter(test)

In [18]:
sent = getter_df_train.get_next()
print(sent)

[('Obesity', 'NN', 'O'), ('in', 'IN', 'O'), ('Low-', 'NNP', 'O'), ('and', 'CC', 'O'), ('Middle-Income', 'thing', 'O'), ('Countries', 'NNPS', 'O'), (':', ':', 'O'), ('Burden', 'NNP', 'O'), (',', ',', 'O'), ('Drivers', 'NNP', 'O'), (',', ',', 'O'), ('and', 'CC', 'O'), ('Emerging', 'VBG', 'O'), ('Challenges', 'NNS', 'O'), ('.', '.', 'O')]


In [19]:
sentences_train = getter_df_train.sentences
sentences_test = getter_df_test.sentences

In [20]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
#         'no_of_capital' : sum(1 for c in word if c.isupper()),
#         'hyphen' : word.count('-'),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [21]:
X_train = [sent2features(s) for s in sentences_train]
y_train = [sent2labels(s) for s in sentences_train]

KeyboardInterrupt: 

In [ ]:
X_test = [sent2features(s) for s in sentences_test]

In [25]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [26]:
y_pred = crf.predict(X_test)

In [27]:
result = []
for i in y_pred:
    for j in i:
        result.append(j)

In [37]:
def bio_tagger(ne_tagged):
    bio_tagged = []
    prev_tag = "O"
    
    for tag in ne_tagged:
        if tag == "O": #O
            bio_tagged.append(tag)
            prev_tag = tag
            continue
            
        if tag != "O" and prev_tag == "O": # Begin NE
            bio_tagged.append("B-"+'indications')
            prev_tag = tag
            
        elif prev_tag != "O" and prev_tag == tag: # Inside NE
            bio_tagged.append("I-"+'indications')
            prev_tag = tag
            
    return bio_tagged

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O'

In [39]:
submit = pd.read_csv('sample_submission-3.csv')
submit['tag'] = bio_tagger(result)

In [40]:
submit.to_csv('POS_tagged_1.csv',index = False)

In [8]:
import pandas as pd
df_doc_useful = pd.read_csv('train.csv')

In [3]:
with open('only_useful_doc_train_ner.txt','w') as file:
    for i in range(df_doc_useful.shape[0]):
        file.write(df_doc_useful['Word'][i]+'\t'+df_doc_useful['tag'][i])
        file.write('\n')

In [11]:
df_doc_useful.fillna('strange',inplace = True)

In [ ]:
with open('all_doc_ner.txt','w') as file:
    for doc in df_doc_useful['Doc_ID'].unique():
        sub_df = df_doc_useful[df_doc_useful['Doc_ID']==doc]
        for i in range(sub_df.shape[0]):
            file.write(list(sub_df['Word'])[i]+'\t'+list(sub_df['tag'])[i])
            file.write('\n')
        file.write('\n')

In [ ]:
with open('all_doc_ner.txt','w') as file:
    for doc in df_doc_useful['Doc_ID'].unique():
        sub_df = df_doc_useful[df_doc_useful['Doc_ID']==doc]
        for i in range(sub_df.shape[0]):
            file.write(list(sub_df['Word'])[i]+'\t'+list(sub_df['tag'])[i])
            file.write('\n')
        file.write('\n')